# This notebook further optimize the simulation efficiency by vectorizing the measurement generation & gradient calc and update. Only ConsensusEKF is still performed via a subloop due to its stateful nature, where vectorization is difficult.

**Also, we only consider fixed-topology networks in the experiments below.**

In [1]:

%matplotlib inline
import numpy as np
from matplotlib import pyplot as plt
from matplotlib import animation, rc

import pickle as pkl
import networkx as nx
from matplotlib import style
from functools import partial

from utils.dLdp import analytic_dLdp,analytic_dhdz,analytic_dhdq
from utils.ConsensusEKF import ConsensusEKF


%load_ext autoreload
%autoreload 2

## Set up F_tilde data structure

In [2]:
def circulant(i,q,p,prev,post,undirected=False):
    """
        Generate a circulant graph with len(p) nodes, node i connected with [i-prev:i+post],i-prev and i+post included but self-loop eliminated.
    """
    n = len(p)
    G = nx.DiGraph()
    edges = [(j%n,i) for i in range(n) for j in range(i-prev,i+post+1)]
    G.add_edges_from(edges)
    G.remove_edges_from(nx.selfloop_edges(G))
    if undirected:
        G = G.to_undirected()
    return G
    

In [6]:
def single_meas_func(C1,C0,k,b,dist):
    return k*(dist-C1)**b+C0

def joint_meas_func(C1s,C0s,ks,bs,x,ps):

    # Casting for the compatibility of jax.numpy

    C1=np.array(C1s)
    C0=np.array(C0s)
    k=np.array(ks)
    b=np.array(bs)
    p=np.array(ps)

    # Keep in mind that x is a vector of [q,q'], thus only the first half of components are observable.    
    dists=np.linalg.norm(x[:len(x)//2]-p,axis=1)

    return single_meas_func(C1,C0,k,b,dists) 

In [ ]:
N_sen = 6 # Number of sensors.

p_0 = np.random.rand(N_sen,2)*3
qhat_0 = np.random.rand(N_sen,2)*10
q = np.array([6,6])

comm_network_generator=lambda i,q,p:circulant(i,q,p,prev=1,post=0,undirected=True)

N_iter=100
C_gain=0.1
coordinate=False

# Set up virtual sensors
C1=-0.3 # Setting C1 as a negative number mitigates the blowing-up effect when the sensors are close to the source.
C0=0
k=1
b=-2
noise_std = 0.01
minimum_sensing_reading=1e-5

# Data containers
p = np.array(p_0) # Sensor Positins
qhat = np.array(qhat_0)



# Create the list of single-term partial FIM's.
def F_single(dh,qhat,ps):
    A = dh(qhat,ps)
    return A.T.dot(A)

def joint_F_single(qhat,ps): # Verified to be correct.
    # The vectorized version of F_single.
    # The output shape is (N_sensor, q_dim, q_dim).
    # Where output[i]=F_single(dh,qhat,ps[i])
    A = analytic_dhdq(qhat,ps,C1s=C1,C0s=C0,ks=k,bs=b)
    return A[:,np.newaxis,:]*A[:,:,np.newaxis]


# Create the list local estimate of global FIM.
F_0 = joint_F_single(qhat,p)
F = np.array(F_0)
F_est = F+1e-8*np.eye(2) # Adding a small I to ensure invertibility

# Create the communication network and consensus weight matrix.
G = comm_network_generator(0,q,p)
estimators = [ConsensusEKF(q_0,C_gain=C_gain) for q_0 in qhat_0]
A = np.array(nx.adj_matrix(G).todense().astype(float))
A +=np.eye(len(A))

W = A/np.sum(A,axis=1) # The weight matrix required by parallel two-pass algorithm.

# The initialization of local measurement functions and the derivative functions. 
# This is not very pretty. But is required by Consensus EKF.
hs = []
dhdzs = []
dhdqs = []
C1s=C1*np.ones(N_sen)
C0s = C0*np.ones(N_sen)
ks = k * np.ones(N_sen)
bs = b*np.ones(N_sen)


d = np.zeros(N_sen)
for i in G.nodes():  
    N_i = [i]+list(G[i])     
    C1s_i=C1s[N_i]
    C0s_i = C0s[N_i]
    ks_i = ks[N_i]
    bs_i = bs[N_i]
    hs.append(partial(joint_meas_func,C1s_i,C0s_i,ks_i,bs_i))# Freeze the coefficients, the signature becomes h(z,ps))
    dhdzs.append(partial(analytic_dhdz,C1s=C1s_i,C0s=C0s_i,ks=ks_i,bs=bs_i))
    dhdqs.append(partial(analytic_dhdq,C1s=C1s_i,C0s=C0s_i,ks=ks_i,bs=bs_i))
    d[i]=len(N_i)

# Variables for parallel two-pass algorithm.
inv_d = 1/d
w_F_est = F_est/d[:,np.newaxis,np.newaxis]
    
import time

t=time.time()



# Enter main loop
for _ in range(N_iter):
    # Measure
    r = np.linalg.norm(q-p,axis=1)
    y = k* ((r-C1)**b)+C0 + np.random.randn(N_sen)*noise_std
    y[y<=0]=minimum_sensing_reading # We don't want y to be zero or negative.
    
    
    # Estimate
    zhats = np.array([est.z for est in estimators])
    for i in G.nodes():
        N_i = [i]+list(G[i]) 
        # Estimate
        qhat[i,:]=estimators[i].update_and_estimate_loc(hs[i],dhdzs[i],y[N_i],p[N_i],zhats[N_i])
    #print(np.linalg.norm(qhat-q))
    
    # Partial FIM Calculation
    for _ in range(5):
        new_F = joint_F_single(qhat,p)
        dF = new_F-F
        F=new_F

        # FIM Consensus using parallel two-pass algorithm
        inv_d = W.dot(inv_d)
        w_F_est = (w_F_est.T.dot(W)).T + dF/d[:,np.newaxis,np.newaxis]
        F_est = w_F_est/inv_d[:,np.newaxis,np.newaxis]   
    
    # Gradient update
    
print('Time:',time.time()-t)

In [124]:
print(F_est)

[[[3.47068727e-05 4.14680612e-05]
  [4.14680612e-05 5.04195518e-05]]

 [[3.61737056e-05 4.21559486e-05]
  [4.21559486e-05 5.07791037e-05]]

 [[3.38298239e-05 4.04855950e-05]
  [4.04855950e-05 5.02448466e-05]]

 [[3.37645436e-05 3.75336216e-05]
  [3.75336216e-05 4.38092207e-05]]

 [[3.22977107e-05 3.68457342e-05]
  [3.68457342e-05 4.34496688e-05]]

 [[3.46415925e-05 3.85160879e-05]
  [3.85160879e-05 4.39839259e-05]]]


In [125]:
ave_F = np.average(joint_F_single(qhat,p),axis=0)

print(ave_F)

[[3.42257082e-05 3.95008414e-05]
 [3.95008414e-05 4.71043863e-05]]


In [126]:
true_F=np.average(joint_F_single(q,p),axis=0)

print(true_F)

[[2.91598886e-05 3.41824632e-05]
 [3.41824632e-05 4.06588571e-05]]
